In [1]:
!pip install schedule

In [2]:
%run ../spark_session.ipynb

In [3]:
import time
import requests
import schedule
from pyspark.sql.functions import explode, col

In [4]:
def explode_data(df):
    df = df.withColumn("l", explode(df.l))
    return df.withColumn("vs", explode(df.l.vs))

In [5]:
def output_formatter(df):
    df = df.withColumnRenamed("hr", "hora") \
                  .withColumn("codigo_empresa", col("l.c")) \
                  .withColumn("codigo_linha", col("l.cl")) \
                  .withColumn("sentido_operacao", col("l.sl")) \
                  .withColumn("destino_linha", col("l.lt0")) \
                  .withColumn("origem_linha", col("l.lt1")) \
                  .withColumn("veiculos_circulacao", col("l.qv")) \
                  .withColumn("prefixo_veiculo", col("vs.p")) \
                  .withColumn("flag_acessivel", col("vs.a")) \
                  .withColumn("data_hora", col("vs.ta")) \
                  .withColumn("latitude", col("vs.py")) \
                  .withColumn("longitude", col("vs.px"))

    return df.drop("l","vs") 

In [6]:
def job_raw_position():
    print("Criando sessão spark e carregando dados /raw/position")
    spark = get_or_create_session_spark("raw_trusted_position")
    df = spark.read.format("delta").load("s3a://raw/position")
    
    print("Aplicando tratamentos")
    df = explode_data(df)
    df = output_formatter(df)
    
    print("Salvando dados! /trusted/position")
    df.write.format("delta").mode("overwrite").save("s3a://trusted/position")
    
    spark.stop()
    print("Processamento finalizado!")

In [7]:
schedule.every(5).minutes.do(job_raw_position)

while True:
    schedule.run_pending()
    time.sleep(1)

Criando sessão spark e carregando dados /raw/position
Obtendo sessão spark raw_trusted_position
Sessão Spark obtida com sucesso! raw_trusted_position
Aplicando tratamentos
Salvando dados! /trusted/position


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Jav

Py4JError: An error occurred while calling o85.save